In [ ]:
%pip install pandas nltk

In [1]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [1]:
import pandas as pd

data = {
    "review": [
        "This product is amazing! I love the quality and feel.",
        "Terrible experience. The product broke in two days.",
        "Great fit and comfortable to wear all day.",
        "Not worth the price. Disappointed with the purchase."
    ]
}

reviews = pd.DataFrame(data)
print(reviews)


                                              review
0  This product is amazing! I love the quality an...
1  Terrible experience. The product broke in two ...
2         Great fit and comfortable to wear all day.
3  Not worth the price. Disappointed with the pur...


In [2]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk

# nltk.download("punkt_tab")
# nltk.download("stopwords")
# nltk.download("wordnet")
# nltk.download("punkt")


def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r"[^a-z\s]", "", text)  # Remove special characters
    tokens = word_tokenize(text)  # Tokenize
    tokens = [
        t for t in tokens if t not in stopwords.words("english")
    ]  # Remove stopwords
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(t) for t in tokens]  # Lemmatize
    return " ".join(tokens)


reviews["cleaned_review"] = reviews["review"].apply(clean_text)
print(reviews)

                                              review  \
0  This product is amazing! I love the quality an...   
1  Terrible experience. The product broke in two ...   
2         Great fit and comfortable to wear all day.   
3  Not worth the price. Disappointed with the pur...   

                              cleaned_review  
0          product amazing love quality feel  
1  terrible experience product broke two day  
2             great fit comfortable wear day  
3          worth price disappointed purchase  


In [7]:
%pip install textblob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 624.3/624.3 kB 4.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from textblob import TextBlob


def sentiment_analysis(text):
    analysis = TextBlob(text)
    return "positive" if analysis.sentiment.polarity > 0 else "negative"


reviews["sentiment"] = reviews["cleaned_review"].apply(sentiment_analysis)
print(reviews)

                                              review  \
0  This product is amazing! I love the quality an...   
1  Terrible experience. The product broke in two ...   
2         Great fit and comfortable to wear all day.   
3  Not worth the price. Disappointed with the pur...   

                              cleaned_review sentiment  
0          product amazing love quality feel  positive  
1  terrible experience product broke two day  negative  
2             great fit comfortable wear day  positive  
3          worth price disappointed purchase  negative  


In [ ]:
from textblob import TextBlob


def sentiment_analysis(text):
    analysis = TextBlob(text)
    return "positive" if analysis.sentiment.polarity > 0 else "negative"


reviews["sentiment"] = reviews["cleaned_review"].apply(sentiment_analysis)
print(reviews)

In [4]:
positive_reviews = reviews[reviews["sentiment"] == "positive"]
negative_reviews = reviews[reviews["sentiment"] == "negative"]

print("Positive Reviews:")
print(positive_reviews["review"])
print("\nNegative Reviews:")
print(negative_reviews["review"])

Positive Reviews:
0    This product is amazing! I love the quality an...
2           Great fit and comfortable to wear all day.
Name: review, dtype: object

Negative Reviews:
1    Terrible experience. The product broke in two ...
3    Not worth the price. Disappointed with the pur...
Name: review, dtype: object


In [14]:
pip install tensorflow

  Using cached setuptools-75.8.0-py3-none-any.whl.metadata (6.7 kB)
  Using cached numpy-1.26.4-cp312-cp312-macosx_10_9_x86_64.whl.metadata (61 kB)
  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
  Using cached rich-13.9.4-py3-none-any.whl.metadata (18 kB)
  Using cached MarkupSafe-3.0.2-cp312-cp312-macosx_10_13_universal2.whl.metadata (4.0 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.7/259.7 MB 20.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 38.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 39.4 MB/s eta 0:00:0000:0100:01
Using cached numpy-1.26.4-cp312-cp312-macosx_10_9_x86_64.whl (20.3 MB)
   ━━━━━━━━━

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google/pegasus-xsum")
model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-xsum")

def summarize_reviews(reviews):
    inputs = tokenizer(" ".join(reviews), return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs.input_ids, max_length=50, min_length=25, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Summarize positive reviews
positive_summary = summarize_reviews(positive_reviews["review"].tolist())
print("Positive Summary:")
print(positive_summary)

# Summarize negative reviews
negative_summary = summarize_reviews(negative_reviews["review"].tolist())
print("Negative Summary:")
print(negative_summary)

/Users/shreyasbattula/IR/IR/lib/python3.12/site-packages/huggingface_hub/file_download.py:651: UserWarning: Not enough free disk space to download the file. The expected file size is: 2275.33 MB. The target location /Users/shreyasbattula/.cache/huggingface/hub/models--google--pegasus-xsum/blobs only has 2122.26 MB free disk space.
  warnings.warn(


KeyboardInterrupt: 

In [7]:
from transformers import pipeline

# Load summarization pipeline
summarizer = pipeline(
    "summarization", model="facebook/bart-large-cnn"
)  # Use an equivalent pre-trained model

# Summarize positive reviews
positive_summary = summarizer(
    " ".join(positive_reviews["review"].tolist()),
    max_length=50,
    min_length=25,
    do_sample=False,
)
print("Positive Summary:")
print(positive_summary[0]["summary_text"])

# Summarize negative reviews
negative_summary = summarizer(
    " ".join(negative_reviews["review"].tolist()),
    max_length=50,
    min_length=25,
    do_sample=False,
)
print("Negative Summary:")
print(negative_summary[0]["summary_text"])

All PyTorch model weights were used when initializing TFBartForConditionalGeneration.

All the weights of TFBartForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.
Device set to use 0
Your max_length is set to 50, but your input_length is only 23. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=11)
Your max_length is set to 50, but your input_length is only 24. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=12)


Positive Summary:
This product is amazing! I love the quality and feel. Great fit and comfortable to wear all day. I will be wearing this to work every day.
Negative Summary:
The product broke in two days. Not worth the price. Disappointed with the purchase. Terrible experience. The product broke two days after it was purchased.


In [2]:
pip install torch

  Using cached torch-2.2.2-cp312-none-macosx_10_9_x86_64.whl.metadata (25 kB)
  Using cached sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached jinja2-3.1.5-py3-none-any.whl.metadata (2.6 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached torch-2.2.2-cp312-none-macosx_10_9_x86_64.whl (150.8 MB)
Using cached jinja2-3.1.5-py3-none-any.whl (134 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 3.1 MB/s eta 0:00:00a 0:00:01
Using cached sympy-1.13.3-py3-none-any.whl (6.2 MB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
Positive Summary:
"The product is amazing with great quality, fit, and comfort."

Negative Summary:
"Customers were disappointed with the price and durability."
